In [36]:
## import

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import scale
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor as RFR
from bayes_opt import BayesianOptimization as BO
from IPython.display import display
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
import xgboost as XGB
from sklearn.ensemble import GradientBoostingRegressor as GBR
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn import linear_model

%matplotlib inline
plt.style.use('ggplot')


In [6]:
df_train = pd.read_csv('train_new_features.csv', index_col='id')
df_train.head()

,sg,n_atoms,x_Al,x_Ga,x_In,a,b,c,alpha,beta,...,avg_size_calc,avg_EA,avg_EN,avg_HOMO,avg_IP,avg_LUMO,avg_mass,avg_rd_max,avg_rp_max,avg_rs_max
id,,,,,,,,,,,,,,,,,,,,,
1,33,80.0,0.6250,0.3750,0.000,9.9523,8.5513,9.1775,90.0026,90.0023,...,0.201799,-0.020559,0.172588,-0.277587,-0.593489,0.028551,4.405298,0.282055,0.147109,0.110364
2,194,80.0,0.6250,0.3750,0.000,6.1840,6.1838,23.6287,90.0186,89.9980,...,0.201425,-0.020521,0.172268,-0.277073,-0.592391,0.028498,4.397146,0.281534,0.146837,0.110160
3,227,40.0,0.8125,0.1875,0.000,9.7510,5.6595,13.9630,90.9688,91.1228,...,0.184712,-0.023364,0.168444,-0.276418,-0.591693,0.033063,3.578027,0.299762,0.150104,0.112850
4,167,30.0,0.7500,0.0000,0.250,5.0036,5.0034,13.5318,89.9888,90.0119,...,0.223062,-0.027645,0.168980,-0.278012,-0.584846,0.045990,5.004543,0.288110,0.150574,0.114528
5,194,80.0,0.0000,0.6250,0.375,6.6614,6.6612,24.5813,89.9960,90.0006,...,0.253691,-0.015645,0.152321,-0.233002,-0.483778,0.028951,7.336285,0.175926,0.114532,0.087010


In [86]:
## data
exclude_feature = ['alpha_r', 'beta_r', 'gamma_r', 'x_In', 'x_Al', 'sg', 'n_atoms', 'alpha', 'beta', 'gamma', 'x_Ga', 'vol']
data_train = df_train.drop(exclude_feature, axis=1)
features = list(data_train.drop(['Ef', 'Eg'], axis=1))

X = data_train[features].values
y_g = data_train['Eg'].values
y_f = data_train['Ef'].values

df_test = pd.read_csv('test_new_features.csv', index_col='id')
X_test = df_test.drop(exclude_feature, axis=1).values

In [87]:
## define scoring fuction rmsle
def rmsle(actual, predicted):
    return np.square(np.log(predicted + 1) - np.log(actual + 1)).mean() ** 0.5

my_score = make_scorer(rmsle, greater_is_better=False)

In [96]:
def opt_model(X, y, model, params, cv_search):
    '''
    model: any ml models;
    params: parameters corresponding to the model;
    cv_search: cross validation searching method;
    '''
    if cv_search == "GridSearchCV":
        method = GridSearchCV(model,
                            param_grid=params, 
                            scoring=my_score,
                            cv=5,
                            n_jobs=-1,
                            return_train_score=False,
                            verbose=5)
    
    elif cv_search == "BayesSearchCV":
        method = BayesSearchCV(model, 
                            search_spaces=params,
                            scoring=my_score,
                            cv=5,
                            n_iter=50,
                            n_jobs=-1,
                            return_train_score=False,
                            verbose=5)
         
    opt = method.fit(X, y)
    return opt

### Polynomial ridge regression

In [18]:
model = Pipeline([('poly', PolynomialFeatures()), 
                  ('ridge', linear_model.Ridge())])
param_grid = dict(poly__degree=[3], ridge__alpha=[0.001, 0.005, 0.01, 0.05, 0.1, 0.5])
prr_g = opt_model(X, y_g, model, param_grid, "GridSearchCV")

In [19]:
pd.DataFrame(prr_g.cv_results_)

,mean_fit_time,mean_score_time,mean_test_score,param_poly__degree,param_ridge__alpha,params,rank_test_score,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,std_fit_time,std_score_time,std_test_score
0,0.441281,0.047071,-0.088784,3,0.001,"{'ridge__alpha': 0.001, 'poly__degree': 3}",1,-0.084362,-0.090244,-0.094591,-0.081379,-0.093342,0.022706,0.013641,0.005119
1,0.440321,0.030508,-0.089433,3,0.005,"{'ridge__alpha': 0.005, 'poly__degree': 3}",2,-0.084747,-0.090125,-0.094570,-0.083153,-0.094569,0.097504,0.004008,0.004788
2,0.307908,0.028974,-0.089622,3,0.01,"{'ridge__alpha': 0.01, 'poly__degree': 3}",3,-0.085145,-0.090125,-0.094181,-0.083943,-0.094716,0.020206,0.006058,0.004456
3,0.274383,0.029895,-0.089633,3,0.05,"{'ridge__alpha': 0.05, 'poly__degree': 3}",4,-0.086019,-0.090233,-0.091947,-0.085475,-0.094488,0.012977,0.004040,0.003454
4,0.279791,0.029573,-0.089656,3,0.1,"{'ridge__alpha': 0.1, 'poly__degree': 3}",5,-0.086304,-0.090308,-0.091035,-0.085925,-0.094707,0.011152,0.002547,0.003256
5,0.244887,0.025317,-0.090101,3,0.5,"{'ridge__alpha': 0.5, 'poly__degree': 3}",6,-0.086923,-0.090625,-0.090573,-0.086705,-0.095678,0.046877,0.006863,0.003263


In [20]:
prr_f = opt_model(X, y_f, model, param_grid, "GridSearchCV")

In [21]:
pd.DataFrame(prr_f.cv_results_)

,mean_fit_time,mean_score_time,mean_test_score,param_poly__degree,param_ridge__alpha,params,rank_test_score,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,std_fit_time,std_score_time,std_test_score
0,0.367902,0.032337,-0.034727,3,0.001,"{'ridge__alpha': 0.001, 'poly__degree': 3}",1,-0.035140,-0.032987,-0.035461,-0.033966,-0.036082,0.033698,0.001986,0.001109
1,0.472227,0.038898,-0.034837,3,0.005,"{'ridge__alpha': 0.005, 'poly__degree': 3}",2,-0.035154,-0.033328,-0.035599,-0.034248,-0.035855,0.050211,0.022956,0.000932
2,0.451020,0.029073,-0.034930,3,0.01,"{'ridge__alpha': 0.01, 'poly__degree': 3}",3,-0.035202,-0.033487,-0.035663,-0.034373,-0.035925,0.042346,0.003590,0.000894
3,0.337772,0.035890,-0.035044,3,0.05,"{'ridge__alpha': 0.05, 'poly__degree': 3}",4,-0.035284,-0.033745,-0.035803,-0.034620,-0.035766,0.037549,0.006127,0.000778
4,0.378260,0.028157,-0.035093,3,0.1,"{'ridge__alpha': 0.1, 'poly__degree': 3}",5,-0.035298,-0.033807,-0.035910,-0.034730,-0.035722,0.046924,0.003734,0.000760
5,0.263469,0.030631,-0.035359,3,0.5,"{'ridge__alpha': 0.5, 'poly__degree': 3}",6,-0.035411,-0.034036,-0.036313,-0.035047,-0.035987,0.034914,0.012637,0.000794


In [25]:
best_g = prr_g.best_estimator_
best_f = prr_f.best_estimator_

df_test['bandgap_energy_ev'] = best_g.fit(X, y_g).predict(X_test)
df_test['formation_energy_ev_natom'] = best_f.fit(X, y_f).predict(X_test)
df_pred = df_test[['formation_energy_ev_natom', 'bandgap_energy_ev']]
df_pred.to_csv('submission_linear.csv')
df_test.head()

,sg,n_atoms,x_Al,x_Ga,x_In,a,b,c,alpha,beta,...,avg_EN,avg_HOMO,avg_IP,avg_LUMO,avg_mass,avg_rd_max,avg_rp_max,avg_rs_max,bandgap_energy_ev,formation_energy_ev_natom
id,,,,,,,,,,,,,,,,,,,,,
1,33,80.0,0.1875,0.4688,0.3438,10.5381,9.0141,9.6361,89.9997,90.0003,...,0.154028,-0.239786,-0.499489,0.032240,6.748878,0.197757,0.120840,0.091832,1.599868,0.193111
2,33,80.0,0.7500,0.2500,0.0000,9.8938,8.5014,9.1298,90.0038,90.0023,...,0.172935,-0.281879,-0.603143,0.032139,3.924055,0.299251,0.151839,0.114075,3.811998,0.055712
3,167,30.0,0.6667,0.1667,0.1667,4.9811,4.9808,13.4799,89.9900,90.0109,...,0.173173,-0.281496,-0.595236,0.039659,5.049802,0.285567,0.150554,0.113952,3.411230,0.168182
4,12,80.0,0.5625,0.4375,0.0000,24.3370,6.0091,5.7620,89.9995,103.8581,...,0.165989,-0.265222,-0.566828,0.025803,4.466880,0.263468,0.139404,0.104507,2.986742,0.034201
5,12,80.0,0.1875,0.5000,0.3125,24.6443,6.2906,6.1589,90.0000,104.5929,...,0.152652,-0.237376,-0.495524,0.030406,6.562892,0.196483,0.119535,0.090693,1.646634,0.113410


In [49]:
print("Bandgap RMSLE: {:.6f}, formation RMSLE: {:.6f}".format(abs(prr_g.best_score_), abs(prr_f.best_score_)))
print("Average RMSLE: {:.6f}".format(abs(prr_g.best_score_ + prr_f.best_score_)/2))

Bandgap RMSLE: 0.088784, formation RMSLE: 0.034727
Average RMSLE: 0.061755


### SVR

In [42]:
X_scale = scale(X)
model = SVR()
param_grid = {'kernel':('linear', 'rbf'), 'C':[0.01, 0.1, 1, 10, 100]}

svr_g = opt_model(X_scale, y_g, model, param_grid, "GridSearchCV")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] C=0.01, kernel=linear ...........................................
[CV] C=0.01, kernel=linear ...........................................
[CV] C=0.01, kernel=linear ...........................................
[CV] C=0.01, kernel=linear ...........................................
[CV] . C=0.01, kernel=linear, score=-0.1237067419516988, total=   0.2s
[CV] C=0.01, kernel=linear ...........................................
[CV]  C=0.01, kernel=linear, score=-0.11876351014079427, total=   0.2s
[CV]  C=0.01, kernel=linear, score=-0.13506228467655024, total=   0.2s
[CV] C=0.01, kernel=rbf ..............................................
[CV] C=0.01, kernel=rbf ..............................................
[CV]  C=0.01, kernel=linear, score=-0.12258517866596892, total=   0.2s
[CV] C=0.01, kernel=rbf ..............................................
[CV]  C=0.01, kernel=linear, score=-0.12305367013581676, total=   0.2s
[CV] C=0.01, ker

[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.9s


[CV] . C=0.1, kernel=linear, score=-0.10951340302430912, total=   0.3s
[CV] C=0.1, kernel=rbf ...............................................
[CV] .... C=0.1, kernel=rbf, score=-0.10542142184060393, total=   0.2s
[CV] C=0.1, kernel=rbf ...............................................
[CV] . C=0.1, kernel=linear, score=-0.10961560072794521, total=   0.3s
[CV] C=0.1, kernel=rbf ...............................................
[CV] . C=0.1, kernel=linear, score=-0.10988312667132795, total=   0.4s
[CV] C=0.1, kernel=rbf ...............................................
[CV] .... C=0.1, kernel=rbf, score=-0.10764451092260154, total=   0.2s
[CV] C=1, kernel=linear ..............................................
[CV] .... C=0.1, kernel=rbf, score=-0.10231578366962628, total=   0.2s
[CV] C=1, kernel=linear ..............................................
[CV] .... C=0.1, kernel=rbf, score=-0.09183033069605366, total=   0.2s
[CV] C=1, kernel=linear ..............................................
[CV] .

[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.1min finished


In [43]:
pd.DataFrame(svr_g.cv_results_)

,mean_fit_time,mean_score_time,mean_test_score,param_C,param_kernel,params,rank_test_score,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,std_fit_time,std_score_time,std_test_score
0,0.204214,0.020164,-0.124634,0.01,linear,"{'C': 0.01, 'kernel': 'linear'}",9,-0.123707,-0.135062,-0.118764,-0.122585,-0.123054,0.004967,0.003221,0.005491
1,0.230110,0.043151,-0.141153,0.01,rbf,"{'C': 0.01, 'kernel': 'rbf'}",10,-0.138370,-0.150462,-0.147638,-0.124414,-0.144879,0.012975,0.008460,0.009279
2,0.318818,0.014385,-0.113361,0.1,linear,"{'C': 0.1, 'kernel': 'linear'}",8,-0.116183,-0.121611,-0.109513,-0.109616,-0.109883,0.037118,0.000814,0.004836
3,0.184239,0.033198,-0.101962,0.1,rbf,"{'C': 0.1, 'kernel': 'rbf'}",4,-0.105421,-0.107645,-0.102316,-0.091830,-0.102598,0.010361,0.010828,0.005430
4,1.165233,0.020310,-0.112196,1,linear,"{'C': 1, 'kernel': 'linear'}",7,-0.116007,-0.119509,-0.109653,-0.107325,-0.108486,0.102353,0.005713,0.004732
5,0.255603,0.045947,-0.093850,1,rbf,"{'C': 1, 'kernel': 'rbf'}",3,-0.096708,-0.096200,-0.092688,-0.086796,-0.096857,0.054279,0.031227,0.003843
6,4.809436,0.016061,-0.112130,10,linear,"{'C': 10, 'kernel': 'linear'}",5,-0.116095,-0.119373,-0.109472,-0.107260,-0.108448,0.321029,0.002707,0.004744
7,0.348463,0.020264,-0.089802,10,rbf,"{'C': 10, 'kernel': 'rbf'}",1,-0.090638,-0.092085,-0.089724,-0.083618,-0.092943,0.015778,0.001462,0.003287
8,33.298763,0.015914,-0.112136,100,linear,"{'C': 100, 'kernel': 'linear'}",6,-0.116181,-0.119370,-0.109427,-0.107276,-0.108426,6.192916,0.006384,0.004763
9,2.076516,0.024035,-0.091046,100,rbf,"{'C': 100, 'kernel': 'rbf'}",2,-0.092827,-0.092655,-0.093278,-0.083970,-0.092499,0.300631,0.008136,0.003547


In [44]:
svr_f = opt_model(X_scale, y_f, model, param_grid, "GridSearchCV")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] C=0.01, kernel=linear ...........................................
[CV] C=0.01, kernel=linear ...........................................
[CV] C=0.01, kernel=linear ...........................................
[CV] C=0.01, kernel=linear ...........................................
[CV]  C=0.01, kernel=linear, score=-0.059268967766137846, total=   0.1s
[CV]  C=0.01, kernel=linear, score=-0.05858708851238363, total=   0.1s
[CV] C=0.01, kernel=linear ...........................................
[CV] C=0.01, kernel=rbf ..............................................
[CV]  C=0.01, kernel=linear, score=-0.06173714758204844, total=   0.1s
[CV]  C=0.01, kernel=linear, score=-0.057678050151486845, total=   0.1s
[CV] C=0.01, kernel=rbf ..............................................
[CV] C=0.01, kernel=rbf ..............................................
[CV] ... C=0.01, kernel=rbf, score=-0.05318259037690307, total=   0.0s
[CV] C=0.01, k

[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.5s


[CV] . C=0.1, kernel=linear, score=-0.05123497421250436, total=   0.2s
[CV] C=1, kernel=rbf .................................................
[CV] ...... C=1, kernel=rbf, score=-0.05026609357305964, total=   0.1s
[CV] C=1, kernel=rbf .................................................
[CV] ...... C=1, kernel=rbf, score=-0.05004209203388334, total=   0.0s
[CV] C=1, kernel=rbf .................................................
[CV] ..... C=1, kernel=rbf, score=-0.048940339123592906, total=   0.1s
[CV] C=1, kernel=rbf .................................................
[CV] .... C=1, kernel=linear, score=-0.0529735102462802, total=   0.4s
[CV] C=1, kernel=linear ..............................................
[CV] ... C=1, kernel=linear, score=-0.05681864648537425, total=   0.4s
[CV] C=1, kernel=linear ..............................................
[CV] ...... C=1, kernel=rbf, score=-0.05302226254024374, total=   0.1s
[CV] C=10, kernel=linear .............................................
[CV] .

[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   20.5s finished


In [45]:
pd.DataFrame(svr_f.cv_results_)

,mean_fit_time,mean_score_time,mean_test_score,param_C,param_kernel,params,rank_test_score,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,std_fit_time,std_score_time,std_test_score
0,0.127509,0.008958,-0.059223,0.01,linear,"{'C': 0.01, 'kernel': 'linear'}",10,-0.059269,-0.058587,-0.061737,-0.057678,-0.058845,0.016558,0.004592,0.001361
1,0.051821,0.006290,-0.054062,0.01,rbf,"{'C': 0.01, 'kernel': 'rbf'}",7,-0.053183,-0.053705,-0.055584,-0.050863,-0.056975,0.009633,0.001123,0.002095
2,0.177644,0.004470,-0.053342,0.1,linear,"{'C': 0.1, 'kernel': 'linear'}",5,-0.052397,-0.052997,-0.055989,-0.051235,-0.054091,0.013229,0.003101,0.001614
3,0.027105,0.004365,-0.049103,0.1,rbf,"{'C': 0.1, 'kernel': 'rbf'}",1,-0.048188,-0.048672,-0.050336,-0.047389,-0.050928,0.004013,0.001231,0.001328
4,0.385196,0.003438,-0.054006,1,linear,"{'C': 1, 'kernel': 'linear'}",6,-0.052974,-0.052780,-0.056819,-0.051779,-0.055678,0.031276,0.001516,0.001910
5,0.057480,0.004700,-0.050483,1,rbf,"{'C': 1, 'kernel': 'rbf'}",3,-0.050142,-0.050266,-0.050042,-0.048940,-0.053022,0.007966,0.003639,0.001355
6,1.983920,0.002659,-0.054112,10,linear,"{'C': 10, 'kernel': 'linear'}",8,-0.053108,-0.052593,-0.057345,-0.051657,-0.055858,0.161025,0.000506,0.002137
7,0.091938,0.002924,-0.049480,10,rbf,"{'C': 10, 'kernel': 'rbf'}",2,-0.050915,-0.048004,-0.046510,-0.049134,-0.052837,0.010930,0.000174,0.002211
8,8.342992,0.002692,-0.054305,100,linear,"{'C': 100, 'kernel': 'linear'}",9,-0.053204,-0.052880,-0.057442,-0.051892,-0.056106,1.676724,0.001213,0.002105
9,0.330112,0.004428,-0.050853,100,rbf,"{'C': 100, 'kernel': 'rbf'}",4,-0.050065,-0.049997,-0.051965,-0.047646,-0.054593,0.198207,0.001914,0.002318


In [46]:
best_g = prr_g.best_estimator_
best_f = prr_f.best_estimator_

df_test['bandgap_energy_ev'] = best_g.fit(X, y_g).predict(X_test)
df_test['formation_energy_ev_natom'] = best_f.fit(X, y_f).predict(X_test)
df_pred = df_test[['formation_energy_ev_natom', 'bandgap_energy_ev']]
df_pred.to_csv('submission_svr.csv')
df_test.head()

,sg,n_atoms,x_Al,x_Ga,x_In,a,b,c,alpha,beta,...,avg_EN,avg_HOMO,avg_IP,avg_LUMO,avg_mass,avg_rd_max,avg_rp_max,avg_rs_max,bandgap_energy_ev,formation_energy_ev_natom
id,,,,,,,,,,,,,,,,,,,,,
1,33,80.0,0.1875,0.4688,0.3438,10.5381,9.0141,9.6361,89.9997,90.0003,...,0.154028,-0.239786,-0.499489,0.032240,6.748878,0.197757,0.120840,0.091832,1.599868,0.193111
2,33,80.0,0.7500,0.2500,0.0000,9.8938,8.5014,9.1298,90.0038,90.0023,...,0.172935,-0.281879,-0.603143,0.032139,3.924055,0.299251,0.151839,0.114075,3.811998,0.055712
3,167,30.0,0.6667,0.1667,0.1667,4.9811,4.9808,13.4799,89.9900,90.0109,...,0.173173,-0.281496,-0.595236,0.039659,5.049802,0.285567,0.150554,0.113952,3.411230,0.168182
4,12,80.0,0.5625,0.4375,0.0000,24.3370,6.0091,5.7620,89.9995,103.8581,...,0.165989,-0.265222,-0.566828,0.025803,4.466880,0.263468,0.139404,0.104507,2.986742,0.034201
5,12,80.0,0.1875,0.5000,0.3125,24.6443,6.2906,6.1589,90.0000,104.5929,...,0.152652,-0.237376,-0.495524,0.030406,6.562892,0.196483,0.119535,0.090693,1.646634,0.113410


In [48]:
print("Bandgap RMSLE: {:.6f}, formation RMSLE: {:.6f}".format(abs(svr_g.best_score_), abs(svr_f.best_score_)))
print("Average RMSLE: {:.6f}".format(abs((svr_g.best_score_ + svr_f.best_score_)/2)))

Bandgap RMSLE: 0.089802, formation RMSLE: 0.049103
Average RMSLE: 0.069452


In [75]:
## Bayes search
model = SVR()
param_grid = {'kernel': ['linear', 'rbf'],
                'C': (1e-3, 1e+1, 'log-uniform'),
                'degree': (1, 8),
                'gamma': (1e-3, 1e+3, 'log-uniform')}

svr_bs_g = opt_model(X_scale, y_g, model, param_grid, "BayesSearchCV")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] gamma=0.00515818669848, C=0.00150452628636, degree=2, kernel=linear 
[CV] gamma=0.00515818669848, C=0.00150452628636, degree=2, kernel=linear 
[CV] gamma=0.00515818669848, C=0.00150452628636, degree=2, kernel=linear 
[CV] gamma=0.00515818669848, C=0.00150452628636, degree=2, kernel=linear 
[CV]  gamma=0.00515818669848, C=0.00150452628636, degree=2, kernel=linear, score=-0.14723162100557854, total=   0.2s
[CV] gamma=0.00515818669848, C=0.00150452628636, degree=2, kernel=linear 
[CV]  gamma=0.00515818669848, C=0.00150452628636, degree=2, kernel=linear, score=-0.13165737699749328, total=   0.2s
[CV]  gamma=0.00515818669848, C=0.00150452628636, degree=2, kernel=linear, score=-0.13494992270579576, total=   0.3s
[CV] gamma=0.00961033690839, C=2.22408451511, degree=2, kernel=linear 
[CV] gamma=0.00961033690839, C=2.22408451511, degree=2, kernel=linear 
[CV]  gamma=0.00515818669848, C=0.00150452628636, degree=2, kernel=linear, sc

[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    4.3s


[CV]  gamma=0.227734956424, C=0.0997665442768, degree=8, kernel=linear, score=-0.10954288222098474, total=   0.3s
[CV] gamma=0.300118447553, C=3.86200413597, degree=1, kernel=rbf .....
[CV]  gamma=0.227734956424, C=0.0997665442768, degree=8, kernel=linear, score=-0.10960200166348093, total=   0.3s
[CV] gamma=0.300118447553, C=3.86200413597, degree=1, kernel=rbf .....
[CV]  gamma=0.227734956424, C=0.0997665442768, degree=8, kernel=linear, score=-0.10991764927497433, total=   0.3s
[CV] gamma=0.300118447553, C=3.86200413597, degree=1, kernel=rbf .....
[CV]  gamma=0.300118447553, C=3.86200413597, degree=1, kernel=rbf, score=-0.09644432570418526, total=   0.3s
[CV] gamma=0.300118447553, C=3.86200413597, degree=1, kernel=rbf .....
[CV]  gamma=0.300118447553, C=3.86200413597, degree=1, kernel=rbf, score=-0.09429638034117073, total=   0.3s
[CV] gamma=0.300118447553, C=3.86200413597, degree=1, kernel=rbf .....
[CV]  gamma=0.300118447553, C=3.86200413597, degree=1, kernel=rbf, score=-0.094457261

[Parallel(n_jobs=-1)]: Done  18 out of  20 | elapsed:    5.3s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    5.4s finished


Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] gamma=0.00291573888919, C=0.236821381748, degree=4, kernel=linear 
[CV] gamma=0.00291573888919, C=0.236821381748, degree=4, kernel=linear 
[CV] gamma=0.00291573888919, C=0.236821381748, degree=4, kernel=linear 
[CV] gamma=0.00291573888919, C=0.236821381748, degree=4, kernel=linear 
[CV]  gamma=0.00291573888919, C=0.236821381748, degree=4, kernel=linear, score=-0.10802333315958863, total=   0.3s
[CV]  gamma=0.00291573888919, C=0.236821381748, degree=4, kernel=linear, score=-0.11577259293935935, total=   0.4s
[CV] gamma=0.00291573888919, C=0.236821381748, degree=4, kernel=linear 
[CV] gamma=0.00207839106442, C=0.0106087495085, degree=4, kernel=linear 
[CV]  gamma=0.00291573888919, C=0.236821381748, degree=4, kernel=linear, score=-0.10908158638897311, total=   0.4s
[CV]  gamma=0.00291573888919, C=0.236821381748, degree=4, kernel=linear, score=-0.12001087269728136, total=   0.4s
[CV] gamma=0.00207839106442, C=0.0106087495085,

...

[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    3.2s finished


Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] gamma=0.188358184533, C=9.91120633154, degree=7, kernel=rbf .....
[CV] gamma=0.188358184533, C=9.91120633154, degree=7, kernel=rbf .....
[CV] gamma=0.188358184533, C=9.91120633154, degree=7, kernel=rbf .....
[CV] gamma=0.188358184533, C=9.91120633154, degree=7, kernel=rbf .....
[CV]  gamma=0.188358184533, C=9.91120633154, degree=7, kernel=rbf, score=-0.09273620959903886, total=   0.4s
[CV] gamma=0.188358184533, C=9.91120633154, degree=7, kernel=rbf .....
[CV]  gamma=0.188358184533, C=9.91120633154, degree=7, kernel=rbf, score=-0.0959850228512013, total=   0.5s
[CV]  gamma=0.188358184533, C=9.91120633154, degree=7, kernel=rbf, score=-0.09167726078411018, total=   0.5s
[CV] gamma=0.348693495135, C=9.91785529981, degree=2, kernel=rbf .....
[CV] gamma=0.348693495135, C=9.91785529981, degree=2, kernel=rbf .....
[CV]  gamma=0.188358184533, C=9.91120633154, degree=7, kernel=rbf, score=-0.08779756952676093, total=   0.5s
[CV] gam

[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    1.0s remaining:    0.7s


[CV]  gamma=0.348693495135, C=9.91785529981, degree=2, kernel=rbf, score=-0.09420618177072415, total=   0.3s
[CV]  gamma=0.348693495135, C=9.91785529981, degree=2, kernel=rbf, score=-0.10634917323383611, total=   0.3s


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    1.3s finished


In [76]:
pd.DataFrame(svr_bs_g.cv_results_).sort_values("rank_test_score")

,mean_fit_time,mean_score_time,mean_test_score,param_C,param_degree,param_gamma,param_kernel,params,rank_test_score,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,std_fit_time,std_score_time,std_test_score
13,0.319740,0.051871,-0.108238,10.000000,1,0.001000,rbf,"{'gamma': 0.001, 'C': 10.0, 'degree': 1, 'kern...",1,-0.112017,-0.115213,-0.105232,-0.103436,-0.105292,0.022111,0.020504,0.004554
48,0.451064,0.022175,-0.092792,9.911206,7,0.188358,rbf,"{'gamma': 0.188358184533, 'C': 9.91120633154, ...",1,-0.095985,-0.092736,-0.091677,-0.087798,-0.095762,0.039371,0.003850,0.003007
29,0.234808,0.026624,-0.092227,4.506025,1,0.048072,rbf,"{'gamma': 0.0480715450164, 'C': 4.50602503076,...",1,-0.094473,-0.094106,-0.091023,-0.085881,-0.095654,0.016643,0.004339,0.003523
21,0.230434,0.036097,-0.103991,9.074373,1,0.002344,rbf,"{'gamma': 0.00234364110679, 'C': 9.07437259394...",1,-0.108418,-0.109451,-0.101235,-0.098284,-0.102567,0.025840,0.019441,0.004280
32,3.881741,0.013847,-0.112130,10.000000,1,0.001000,linear,"{'gamma': 0.001, 'C': 10.0, 'degree': 1, 'kern...",1,-0.116095,-0.119373,-0.109472,-0.107260,-0.108448,0.224358,0.001126,0.004744
19,3.985724,0.009779,-0.112130,10.000000,8,0.001000,linear,"{'gamma': 0.001, 'C': 10.0, 'degree': 8, 'kern...",1,-0.116095,-0.119373,-0.109472,-0.107260,-0.108448,0.706754,0.003112,0.004744
18,5.710573,0.024537,-0.112130,10.000000,1,1000.000000,linear,"{'gamma': 1000.0, 'C': 10.0, 'degree': 1, 'ker...",1,-0.116095,-0.119373,-0.109472,-0.107260,-0.108448,0.615500,0.011238,0.004744
17,6.476014,0.019013,-0.112130,10.000000,8,1000.000000,linear,"{'gamma': 1000.0, 'C': 10.0, 'degree': 8, 'ker...",1,-0.116095,-0.119373,-0.109472,-0.107260,-0.108448,0.386339,0.005659,0.004744
16,4.574992,0.017218,-0.112130,10.000000,8,1000.000000,linear,"{'gamma': 1000.0, 'C': 10.0, 'degree': 8, 'ker...",1,-0.116095,-0.119373,-0.109472,-0.107260,-0.108448,0.946249,0.003892,0.004744
33,3.794058,0.013120,-0.112130,10.000000,1,0.001000,linear,"{'gamma': 0.001, 'C': 10.0, 'degree': 1, 'kern...",1,-0.116095,-0.119373,-0.109472,-0.107260,-0.108448,0.227499,0.001072,0.004744


In [77]:
svr_bs_f = opt_model(X_scale, y_f, model, param_grid, "BayesSearchCV")
pd.DataFrame(svr_bs_f.cv_results_).sort_values("rank_test_score")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] gamma=0.0267887376017, C=0.0852109996251, degree=4, kernel=linear 
[CV] gamma=0.0267887376017, C=0.0852109996251, degree=4, kernel=linear 
[CV] gamma=0.0267887376017, C=0.0852109996251, degree=4, kernel=linear 
[CV] gamma=0.0267887376017, C=0.0852109996251, degree=4, kernel=linear 
[CV]  gamma=0.0267887376017, C=0.0852109996251, degree=4, kernel=linear, score=-0.052807028044382844, total=   0.1s
[CV]  gamma=0.0267887376017, C=0.0852109996251, degree=4, kernel=linear, score=-0.05267530447069799, total=   0.1s
[CV] gamma=0.0267887376017, C=0.0852109996251, degree=4, kernel=linear 
[CV]  gamma=0.0267887376017, C=0.0852109996251, degree=4, kernel=linear, score=-0.05607355349770488, total=   0.1s
[CV] gamma=78.6241830985, C=0.00297500861415, degree=8, kernel=linear 
[CV] gamma=78.6241830985, C=0.00297500861415, degree=8, kernel=linear 
[CV]  gamma=0.0267887376017, C=0.0852109996251, degree=4, kernel=linear, score=-0.0514173565

[Parallel(n_jobs=-1)]: Done   8 out of  20 | elapsed:    0.3s remaining:    0.5s


[CV] gamma=0.0298555605127, C=4.53642388685, degree=6, kernel=linear .
[CV]  gamma=66.7695134545, C=0.00433914608141, degree=5, kernel=linear, score=-0.06092878570094003, total=   0.1s
[CV]  gamma=66.7695134545, C=0.00433914608141, degree=5, kernel=linear, score=-0.06027338647434701, total=   0.1s
[CV] gamma=0.0298555605127, C=4.53642388685, degree=6, kernel=linear .
[CV]  gamma=0.0298555605127, C=4.53642388685, degree=6, kernel=linear, score=-0.05255563517917863, total=   0.8s
[CV] gamma=0.0298555605127, C=4.53642388685, degree=6, kernel=linear .
[CV]  gamma=0.0298555605127, C=4.53642388685, degree=6, kernel=linear, score=-0.05310940038569719, total=   0.9s
[CV]  gamma=0.0298555605127, C=4.53642388685, degree=6, kernel=linear, score=-0.05184212544974333, total=   0.9s
[CV] gamma=0.0298555605127, C=4.53642388685, degree=6, kernel=linear .
[CV]  gamma=0.0298555605127, C=4.53642388685, degree=6, kernel=linear, score=-0.05726645256672574, total=   0.5s
[CV]  gamma=0.0298555605127, C=4.536

[Parallel(n_jobs=-1)]: Done  18 out of  20 | elapsed:    1.8s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    1.9s finished


Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] gamma=0.0422658954208, C=1.24938474911, degree=7, kernel=rbf ....
[CV] gamma=0.0422658954208, C=1.24938474911, degree=7, kernel=rbf ....
[CV] gamma=0.0422658954208, C=1.24938474911, degree=7, kernel=rbf ....
[CV] gamma=0.0422658954208, C=1.24938474911, degree=7, kernel=rbf ....
[CV]  gamma=0.0422658954208, C=1.24938474911, degree=7, kernel=rbf, score=-0.0497341907494749, total=   0.1s
[CV]  gamma=0.0422658954208, C=1.24938474911, degree=7, kernel=rbf, score=-0.05125109676645571, total=   0.1s
[CV] gamma=0.0422658954208, C=1.24938474911, degree=7, kernel=rbf ....
[CV]  gamma=0.0422658954208, C=1.24938474911, degree=7, kernel=rbf, score=-0.04654466629429064, total=   0.1s
[CV] gamma=0.00950258475003, C=0.0246109293315, degree=1, kernel=linear 
[CV]  gamma=0.0422658954208, C=1.24938474911, degree=7, kernel=rbf, score=-0.04887295035735666, total=   0.1s
[CV] gamma=0.00950258475003, C=0.0246109293315, degree=1, kernel=linear 


...

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] gamma=0.00112333346908, C=0.301484879358, degree=1, kernel=linear 
[CV] gamma=0.00112333346908, C=0.301484879358, degree=1, kernel=linear 
[CV] gamma=0.00112333346908, C=0.301484879358, degree=1, kernel=linear 
[CV] gamma=0.00112333346908, C=0.301484879358, degree=1, kernel=linear 
[CV]  gamma=0.00112333346908, C=0.301484879358, degree=1, kernel=linear, score=-0.05282137188447081, total=   0.1s
[CV] gamma=0.00112333346908, C=0.301484879358, degree=1, kernel=linear 
[CV]  gamma=0.00112333346908, C=0.301484879358, degree=1, kernel=linear, score=-0.056619864602277546, total=   0.2s
[CV] gamma=0.001, C=10.0, degree=1, kernel=rbf .......................
[CV]  gamma=0.00112333346908, C=0.301484879358, degree=1, kernel=linear, score=-0.05148195134104511, total=   0.2s
[CV] gamma=0.001, C=10.0, degree=1, kernel=rbf .......................
[CV]  gamma=0.00112333346908, C=0.301484879358, degree=1, kernel=linear, score=-0.0529383983

[Parallel(n_jobs=-1)]: Done  18 out of  20 | elapsed:    0.8s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.9s finished


Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] gamma=0.82820019505, C=0.237910821392, degree=8, kernel=linear ..
[CV] gamma=0.82820019505, C=0.237910821392, degree=8, kernel=linear ..
[CV] gamma=0.82820019505, C=0.237910821392, degree=8, kernel=linear ..
[CV] gamma=0.82820019505, C=0.237910821392, degree=8, kernel=linear ..
[CV]  gamma=0.82820019505, C=0.237910821392, degree=8, kernel=linear, score=-0.05297903402863172, total=   0.2s
[CV] gamma=0.82820019505, C=0.237910821392, degree=8, kernel=linear ..
[CV]  gamma=0.82820019505, C=0.237910821392, degree=8, kernel=linear, score=-0.05649921966206538, total=   0.1s
[CV]  gamma=0.82820019505, C=0.237910821392, degree=8, kernel=linear, score=-0.05268672866828766, total=   0.2s
[CV]  gamma=0.82820019505, C=0.237910821392, degree=8, kernel=linear, score=-0.051258354333021595, total=   0.2s
[CV] gamma=0.00121570186761, C=0.538408149907, degree=8, kernel=rbf ..
[CV] gamma=0.00121570186761, C=0.538408149907, degree=8, kernel=r

[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.2s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.3s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.4s finished


,mean_fit_time,mean_score_time,mean_test_score,param_C,param_degree,param_gamma,param_kernel,params,rank_test_score,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,std_fit_time,std_score_time,std_test_score
0,0.120372,0.003110,-0.053424,0.085211,4,0.026789,linear,"{'gamma': 0.0267887376017, 'C': 0.085210999625...",1,-0.052675,-0.052807,-0.056074,-0.051417,-0.054144,0.009344,0.000276,0.001582
45,0.070206,0.005139,-0.046242,10.000000,1,0.001000,rbf,"{'gamma': 0.001, 'C': 10.0, 'degree': 1, 'kern...",1,-0.045427,-0.044389,-0.050231,-0.043780,-0.047380,0.011054,0.002036,0.002339
40,0.073396,0.006080,-0.045451,9.735535,8,0.001338,rbf,"{'gamma': 0.00133832653843, 'C': 9.73553511644...",1,-0.044711,-0.043717,-0.048833,-0.043156,-0.046839,0.005561,0.001333,0.002107
38,0.092666,0.003203,-0.049736,9.556544,7,0.071458,rbf,"{'gamma': 0.071458401518, 'C': 9.55654357213, ...",1,-0.051060,-0.048318,-0.046730,-0.049343,-0.053228,0.011472,0.000882,0.002243
34,0.143666,0.003057,-0.053600,0.229913,8,912.068118,linear,"{'gamma': 912.068118399, 'C': 0.229912991822, ...",1,-0.052928,-0.052677,-0.056447,-0.051181,-0.054768,0.010301,0.001299,0.001823
29,0.058157,0.004092,-0.046242,10.000000,8,0.001000,rbf,"{'gamma': 0.001, 'C': 10.0, 'degree': 8, 'kern...",1,-0.045427,-0.044389,-0.050231,-0.043780,-0.047380,0.005353,0.001156,0.002339
27,0.065568,0.001942,-0.054419,0.038536,7,0.001003,linear,"{'gamma': 0.00100281729913, 'C': 0.03853619227...",1,-0.054329,-0.053944,-0.057419,-0.052466,-0.053934,0.011532,0.000491,0.001630
20,0.074043,0.007042,-0.046242,10.000000,1,0.001000,rbf,"{'gamma': 0.001, 'C': 10.0, 'degree': 1, 'kern...",1,-0.045427,-0.044389,-0.050231,-0.043780,-0.047380,0.008082,0.002546,0.002339
17,0.071310,0.004299,-0.046242,10.000000,8,0.001000,rbf,"{'gamma': 0.001, 'C': 10.0, 'degree': 8, 'kern...",1,-0.045427,-0.044389,-0.050231,-0.043780,-0.047380,0.008018,0.001265,0.002339
15,0.069196,0.003717,-0.044084,10.000000,8,0.004927,rbf,"{'gamma': 0.00492706163207, 'C': 10.0, 'degree...",1,-0.043295,-0.042986,-0.045802,-0.042418,-0.045919,0.013396,0.002525,0.001478


In [78]:
print("Bandgap RMSLE: {:.6f}, formation RMSLE: {:.6f}".format(abs(svr_bs_g.best_score_), abs(svr_bs_f.best_score_)))
print("Average RMSLE: {:.6f}".format(abs((svr_bs_g.best_score_ + svr_bs_f.best_score_)/2)))
print("Slightly better than gridsearch ...")

Bandgap RMSLE: 0.090355, formation RMSLE: 0.044084
Average RMSLE: 0.067219
Slightly better than gridsearch ...


### Random forest

In [99]:
## Bayesian Optimization

model = RFR(random_state=99)
param_grid = {"n_estimators": Integer(50, 2000),
     'max_depth': Integer(2, 20),
     'min_samples_split': Integer(2, 15),
     'max_features': Integer(2, 10)}

rfr_g = opt_model(X, y_g, model, param_grid, "BayesSearchCV")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] max_depth=13, n_estimators=1432, max_features=6, min_samples_split=2 
[CV] max_depth=13, n_estimators=1432, max_features=6, min_samples_split=2 
[CV] max_depth=13, n_estimators=1432, max_features=6, min_samples_split=2 
[CV] max_depth=13, n_estimators=1432, max_features=6, min_samples_split=2 
[CV]  max_depth=13, n_estimators=1432, max_features=6, min_samples_split=2, score=-0.09917742436392972, total=  16.1s
[CV] max_depth=13, n_estimators=1432, max_features=6, min_samples_split=2 
[CV]  max_depth=13, n_estimators=1432, max_features=6, min_samples_split=2, score=-0.08687827143272188, total=  16.1s
[CV] max_depth=18, n_estimators=1618, max_features=7, min_samples_split=14 
[CV]  max_depth=13, n_estimators=1432, max_features=6, min_samples_split=2, score=-0.10029765312502774, total=  16.3s
[CV] max_depth=18, n_estimators=1618, max_features=7, min_samples_split=14 
[CV]  max_depth=13, n_estimators=1432, max_features=6, min_

[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   55.9s


[CV]  max_depth=13, n_estimators=1716, max_features=6, min_samples_split=7, score=-0.0994591182512323, total=  18.6s
[CV] max_depth=13, n_estimators=1716, max_features=6, min_samples_split=7 
[CV]  max_depth=13, n_estimators=1716, max_features=6, min_samples_split=7, score=-0.09679774938487251, total=  18.7s
[CV] max_depth=8, n_estimators=1357, max_features=7, min_samples_split=6 
[CV]  max_depth=8, n_estimators=1357, max_features=7, min_samples_split=6, score=-0.09799071381895852, total=  13.1s
[CV] max_depth=8, n_estimators=1357, max_features=7, min_samples_split=6 
[CV]  max_depth=13, n_estimators=1716, max_features=6, min_samples_split=7, score=-0.09294573411775232, total=  17.5s
[CV] max_depth=8, n_estimators=1357, max_features=7, min_samples_split=6 
[CV]  max_depth=13, n_estimators=1716, max_features=6, min_samples_split=7, score=-0.08638720759716637, total=  17.4s
[CV] max_depth=8, n_estimators=1357, max_features=7, min_samples_split=6 
[CV]  max_depth=13, n_estimators=1716, ma

[Parallel(n_jobs=-1)]: Done  18 out of  20 | elapsed:  1.4min remaining:    9.6s


[CV]  max_depth=8, n_estimators=1357, max_features=7, min_samples_split=6, score=-0.08762075360086825, total=  13.2s
[CV]  max_depth=8, n_estimators=1357, max_features=7, min_samples_split=6, score=-0.09443080499397173, total=  12.8s
Fitting 5 folds for each of 4 candidates, totalling 20 fits


...

[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  1.5min finished
/Users/qiuruihao/miniconda2/envs/py3/lib/python3.5/site-packages/skopt/optimizer/optimizer.py:384: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] max_depth=20, n_estimators=152, max_features=10, min_samples_split=14 
[CV] max_depth=20, n_estimators=152, max_features=10, min_samples_split=14 
[CV] max_depth=20, n_estimators=152, max_features=10, min_samples_split=14 
[CV] max_depth=20, n_estimators=152, max_features=10, min_samples_split=14 
[CV]  max_depth=20, n_estimators=152, max_features=10, min_samples_split=14, score=-0.09853144562903327, total=   2.0s
[CV] max_depth=20, n_estimators=152, max_features=10, min_samples_split=14 
[CV]  max_depth=20, n_estimators=152, max_features=10, min_samples_split=14, score=-0.09569201427579725, total=   2.0s
[CV] max_depth=20, n_estimators=2000, max_features=10, min_samples_split=15 
[CV]  max_depth=20, n_estimators=152, max_features=10, min_samples_split=14, score=-0.08591123868016746, total=   2.0s
[CV] max_depth=20, n_estimators=2000, max_features=10, min_samples_split=15 
[CV]  max_depth=20, n_estimators=152, max_feature

[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:   30.3s remaining:   20.2s


[CV]  max_depth=20, n_estimators=2000, max_features=10, min_samples_split=15, score=-0.08588419161904152, total=  27.7s
[CV]  max_depth=20, n_estimators=2000, max_features=10, min_samples_split=15, score=-0.09287032250681344, total=  18.0s


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   48.2s finished


In [102]:
pd.DataFrame(rfr_g.cv_results_).sort_values("rank_test_score")
print(rfr_g.best_estimator_, "\nBest score: {}".format(rfr_g.best_score_))

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=20,
           max_features=10, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=15, min_weight_fraction_leaf=0.0,
           n_estimators=1964, n_jobs=1, oob_score=False, random_state=99,
           verbose=0, warm_start=False) 
Best score: -0.09287458790837982


In [103]:
rfr_f = opt_model(X, y_f, model, param_grid, "BayesSearchCV")
pd.DataFrame(rfr_f.cv_results_).sort_values("rank_test_score")
print(rfr_f.best_estimator_, "\nBest score: {}".format(rfr_f.best_score_))

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] max_depth=15, n_estimators=1423, max_features=9, min_samples_split=11 
[CV] max_depth=15, n_estimators=1423, max_features=9, min_samples_split=11 
[CV] max_depth=15, n_estimators=1423, max_features=9, min_samples_split=11 
[CV] max_depth=15, n_estimators=1423, max_features=9, min_samples_split=11 
[CV]  max_depth=15, n_estimators=1423, max_features=9, min_samples_split=11, score=-0.03621887126086147, total=  19.6s
[CV] max_depth=15, n_estimators=1423, max_features=9, min_samples_split=11 
[CV]  max_depth=15, n_estimators=1423, max_features=9, min_samples_split=11, score=-0.03344303613997202, total=  19.6s
[CV] max_depth=10, n_estimators=864, max_features=10, min_samples_split=8 
[CV]  max_depth=15, n_estimators=1423, max_features=9, min_samples_split=11, score=-0.03717545596355651, total=  19.8s
[CV] max_depth=10, n_estimators=864, max_features=10, min_samples_split=8 
[CV]  max_depth=15, n_estimators=1423, max_features=9

[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   33.8s


[CV]  max_depth=12, n_estimators=94, max_features=3, min_samples_split=9, score=-0.03419158304226256, total=   0.6s
[CV] max_depth=12, n_estimators=94, max_features=3, min_samples_split=9 
[CV]  max_depth=12, n_estimators=94, max_features=3, min_samples_split=9, score=-0.038144452804067974, total=   1.0s
[CV] max_depth=6, n_estimators=1710, max_features=6, min_samples_split=4 
[CV]  max_depth=15, n_estimators=1423, max_features=9, min_samples_split=11, score=-0.03746319129153922, total=  20.8s
[CV] max_depth=6, n_estimators=1710, max_features=6, min_samples_split=4 
[CV]  max_depth=10, n_estimators=864, max_features=10, min_samples_split=8, score=-0.03699275140611922, total=  15.6s
[CV] max_depth=6, n_estimators=1710, max_features=6, min_samples_split=4 
[CV]  max_depth=10, n_estimators=864, max_features=10, min_samples_split=8, score=-0.03244120030939654, total=  15.8s
[CV] max_depth=6, n_estimators=1710, max_features=6, min_samples_split=4 
[CV]  max_depth=6, n_estimators=1710, max_f

[Parallel(n_jobs=-1)]: Done  18 out of  20 | elapsed:   58.5s remaining:    6.5s


[CV]  max_depth=6, n_estimators=1710, max_features=6, min_samples_split=4, score=-0.03869389660478882, total=   9.9s
Fitting 5 folds for each of 4 candidates, totalling 20 fits


...

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] max_depth=11, n_estimators=2000, max_features=10, min_samples_split=2 
[CV] max_depth=11, n_estimators=2000, max_features=10, min_samples_split=2 
[CV] max_depth=11, n_estimators=2000, max_features=10, min_samples_split=2 
[CV] max_depth=11, n_estimators=2000, max_features=10, min_samples_split=2 
[CV]  max_depth=11, n_estimators=2000, max_features=10, min_samples_split=2, score=-0.03591475791117337, total=  28.3s
[CV]  max_depth=11, n_estimators=2000, max_features=10, min_samples_split=2, score=-0.033353281361044304, total=  28.3s
[CV] max_depth=11, n_estimators=2000, max_features=10, min_samples_split=2 
[CV] max_depth=20, n_estimators=161, max_features=10, min_samples_split=2 
[CV]  max_depth=11, n_estimators=2000, max_features=10, min_samples_split=2, score=-0.03764986422554811, total=  28.3s
[CV] max_depth=20, n_estimators=161, max_features=10, min_samples_split=2 
[CV]  max_depth=11, n_estimators=2000, max_features=

[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:   31.2s remaining:   20.8s


[CV]  max_depth=20, n_estimators=161, max_features=10, min_samples_split=2, score=-0.03813404492148323, total=   2.2s
[CV]  max_depth=20, n_estimators=161, max_features=10, min_samples_split=2, score=-0.03485204901922247, total=   2.3s
[CV]  max_depth=11, n_estimators=2000, max_features=10, min_samples_split=2, score=-0.03718678191166376, total=  20.2s


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   48.6s finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=9,
           max_features=9, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=10, min_weight_fraction_leaf=0.0,
           n_estimators=317, n_jobs=1, oob_score=False, random_state=99,
           verbose=0, warm_start=False) 
Best score: -0.03504514263108414


In [104]:
print("Bandgap RMSLE: {:.6f}, formation RMSLE: {:.6f}".format(abs(rfr_g.best_score_), abs(rfr_f.best_score_)))
print("Average RMSLE: {:.6f}".format(abs((rfr_g.best_score_ + rfr_f.best_score_)/2)))

Bandgap RMSLE: 0.092875, formation RMSLE: 0.035045
Average RMSLE: 0.063960


In [107]:
## Predictions
best_g = rfr_g.best_estimator_
best_f = rfr_f.best_estimator_

df_test['bandgap_energy_ev'] = best_g.fit(X, y_g).predict(X_test)
df_test['formation_energy_ev_natom'] = best_f.fit(X, y_f).predict(X_test)
rfr_pred = df_test[['formation_energy_ev_natom', 'bandgap_energy_ev']]
#df_pred.to_csv('submission_rf_bo.csv')

###   Gradient Boosting Regressor (GBR)

In [109]:
model = GBR()
search_params = {
     'learning_rate': Real(0.001, 0.5),
     'n_estimators': Integer(1000, 4000),
     'max_depth': Integer(2, 40),
     'min_samples_split': Integer(2, 15),
     'min_samples_leaf': Integer(2, 50),
     'min_weight_fraction_leaf': Real(0., .5),
     'max_features': Integer(2, 6)
 }

gbr_g = opt_model(X, y_g, model, param_grid, "BayesSearchCV")
display(pd.DataFrame(gbr_g.cv_results_).sort_values("rank_test_score"))
print(gbr_g.best_estimator_, "\nBest score: {}".format(gbr_g.best_score_))

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] max_depth=10, n_estimators=1464, max_features=8, min_samples_split=10 
[CV] max_depth=10, n_estimators=1464, max_features=8, min_samples_split=10 
[CV] max_depth=10, n_estimators=1464, max_features=8, min_samples_split=10 
[CV] max_depth=10, n_estimators=1464, max_features=8, min_samples_split=10 
[CV]  max_depth=10, n_estimators=1464, max_features=8, min_samples_split=10, score=-0.10572819787134699, total=  12.3s
[CV] max_depth=10, n_estimators=1464, max_features=8, min_samples_split=10 
[CV]  max_depth=10, n_estimators=1464, max_features=8, min_samples_split=10, score=-0.1121374369397196, total=  12.4s
[CV] max_depth=12, n_estimators=1632, max_features=5, min_samples_split=13 
[CV]  max_depth=10, n_estimators=1464, max_features=8, min_samples_split=10, score=-0.094925675423309, total=  12.4s
[CV] max_depth=12, n_estimators=1632, max_features=5, min_samples_split=13 
[CV]  max_depth=10, n_estimators=1464, max_features=8,

[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   30.9s


[CV]  max_depth=18, n_estimators=190, max_features=6, min_samples_split=15, score=-0.1032520601400524, total=   2.7s
[CV] max_depth=18, n_estimators=190, max_features=6, min_samples_split=15 
[CV]  max_depth=18, n_estimators=190, max_features=6, min_samples_split=15, score=-0.0939465030230162, total=   2.4s
[CV] max_depth=5, n_estimators=1918, max_features=9, min_samples_split=12 
[CV]  max_depth=18, n_estimators=190, max_features=6, min_samples_split=15, score=-0.09875986951428434, total=   2.5s
[CV] max_depth=5, n_estimators=1918, max_features=9, min_samples_split=12 
[CV]  max_depth=12, n_estimators=1632, max_features=5, min_samples_split=13, score=-0.09831320605295797, total=  11.6s
[CV] max_depth=5, n_estimators=1918, max_features=9, min_samples_split=12 
[CV]  max_depth=12, n_estimators=1632, max_features=5, min_samples_split=13, score=-0.0946837144495339, total=  12.0s
[CV] max_depth=5, n_estimators=1918, max_features=9, min_samples_split=12 
[CV]  max_depth=5, n_estimators=1918

[Parallel(n_jobs=-1)]: Done  18 out of  20 | elapsed:   48.2s remaining:    5.4s


[CV]  max_depth=5, n_estimators=1918, max_features=9, min_samples_split=12, score=-0.09324267052370314, total=   9.6s
[CV]  max_depth=5, n_estimators=1918, max_features=9, min_samples_split=12, score=-0.10049440737695912, total=   9.1s


...

[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:   34.6s remaining:   23.1s


[CV]  max_depth=2, n_estimators=117, max_features=10, min_samples_split=15, score=-0.09611820998196298, total=   0.3s
[CV]  max_depth=2, n_estimators=117, max_features=10, min_samples_split=15, score=-0.09303948798097711, total=   0.2s
[CV]  max_depth=2, n_estimators=117, max_features=10, min_samples_split=15, score=-0.09370309527985851, total=   0.2s
[CV]  max_depth=19, n_estimators=1975, max_features=10, min_samples_split=14, score=-0.09868795308083303, total=  20.7s


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   54.9s finished


,mean_fit_time,mean_score_time,mean_test_score,param_max_depth,param_max_features,param_min_samples_split,param_n_estimators,params,rank_test_score,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,std_fit_time,std_score_time,std_test_score
49,0.253243,0.001800,-0.096262,2,10,15,117,"{'max_depth': 2, 'n_estimators': 117, 'max_fea...",1,-0.097730,-0.100718,-0.096118,-0.093039,-0.093703,0.020714,0.000849,0.002792
25,1.815164,0.016225,-0.096513,2,3,15,1959,"{'max_depth': 2, 'n_estimators': 1959, 'max_fe...",1,-0.096582,-0.099631,-0.099512,-0.089961,-0.096879,0.254119,0.001744,0.003515
15,0.166260,0.000976,-0.096990,2,9,2,85,"{'max_depth': 2, 'n_estimators': 85, 'max_feat...",1,-0.100288,-0.100745,-0.097089,-0.092593,-0.094234,0.027304,0.000122,0.003222
29,1.419772,0.015985,-0.095750,2,2,14,1951,"{'max_depth': 2, 'n_estimators': 1951, 'max_fe...",1,-0.095707,-0.098171,-0.096134,-0.088010,-0.100727,0.081171,0.000947,0.004259
34,1.426885,0.015231,-0.096037,2,2,15,2000,"{'max_depth': 2, 'n_estimators': 2000, 'max_fe...",1,-0.095078,-0.099256,-0.098201,-0.089298,-0.098350,0.067281,0.000232,0.003653
19,3.640823,0.012454,-0.098961,2,10,2,2000,"{'max_depth': 2, 'n_estimators': 2000, 'max_fe...",1,-0.099348,-0.103795,-0.101038,-0.093604,-0.097018,0.063781,0.002412,0.003473
9,0.907498,0.007770,-0.094138,2,4,11,795,"{'max_depth': 2, 'n_estimators': 795, 'max_fea...",1,-0.094178,-0.095024,-0.096061,-0.089368,-0.096060,0.097982,0.002259,0.002487
20,0.296381,0.005188,-0.101154,2,10,2,50,"{'max_depth': 2, 'n_estimators': 50, 'max_feat...",1,-0.104450,-0.105832,-0.102120,-0.095361,-0.098008,0.014114,0.007584,0.003928
39,1.593024,0.020569,-0.100388,3,2,2,1876,"{'max_depth': 3, 'n_estimators': 1876, 'max_fe...",1,-0.103074,-0.106992,-0.100835,-0.090306,-0.100734,0.262374,0.009012,0.005528
43,3.915321,0.013168,-0.098754,2,10,15,2000,"{'max_depth': 2, 'n_estimators': 2000, 'max_fe...",1,-0.101049,-0.103954,-0.098252,-0.091417,-0.099100,0.168602,0.002006,0.004159


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=2, max_features=4,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=11, min_weight_fraction_leaf=0.0,
             n_estimators=795, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False) 
Best score: -0.09413819468529852


In [110]:
gbr_f = opt_model(X, y_f, model, param_grid, "BayesSearchCV")
display(pd.DataFrame(gbr_f.cv_results_).sort_values("rank_test_score"))
print(gbr_f.best_estimator_, "\nBest score: {}".format(gbr_f.best_score_))

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] max_depth=17, n_estimators=1998, max_features=4, min_samples_split=13 
[CV] max_depth=17, n_estimators=1998, max_features=4, min_samples_split=13 
[CV] max_depth=17, n_estimators=1998, max_features=4, min_samples_split=13 
[CV] max_depth=17, n_estimators=1998, max_features=4, min_samples_split=13 
[CV]  max_depth=17, n_estimators=1998, max_features=4, min_samples_split=13, score=-0.04074279689386444, total=  16.8s
[CV] max_depth=17, n_estimators=1998, max_features=4, min_samples_split=13 
[CV]  max_depth=17, n_estimators=1998, max_features=4, min_samples_split=13, score=-0.036787362457032846, total=  16.9s
[CV] max_depth=5, n_estimators=1815, max_features=3, min_samples_split=9 
[CV]  max_depth=17, n_estimators=1998, max_features=4, min_samples_split=13, score=-0.037418532913918966, total=  17.2s
[CV] max_depth=5, n_estimators=1815, max_features=3, min_samples_split=9 
[CV]  max_depth=17, n_estimators=1998, max_features=4

[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   23.3s


[CV]  max_depth=5, n_estimators=1815, max_features=3, min_samples_split=9, score=-0.037725220258935134, total=   3.5s
[CV]  max_depth=6, n_estimators=405, max_features=2, min_samples_split=4, score=-0.03730282366690986, total=   0.7s
[CV] max_depth=6, n_estimators=405, max_features=2, min_samples_split=4 
[CV] max_depth=4, n_estimators=1693, max_features=5, min_samples_split=12 
[CV]  max_depth=5, n_estimators=1815, max_features=3, min_samples_split=9, score=-0.03890551553152623, total=   3.4s
[CV] max_depth=4, n_estimators=1693, max_features=5, min_samples_split=12 
[CV]  max_depth=6, n_estimators=405, max_features=2, min_samples_split=4, score=-0.03917935543230397, total=   0.8s
[CV] max_depth=4, n_estimators=1693, max_features=5, min_samples_split=12 
[CV]  max_depth=4, n_estimators=1693, max_features=5, min_samples_split=12, score=-0.040641007755644123, total=   3.7s
[CV] max_depth=4, n_estimators=1693, max_features=5, min_samples_split=12 
[CV]  max_depth=4, n_estimators=1693, max

[Parallel(n_jobs=-1)]: Done  18 out of  20 | elapsed:   30.7s remaining:    3.4s


[CV]  max_depth=17, n_estimators=1998, max_features=4, min_samples_split=13, score=-0.039819374012953865, total=  14.1s
Fitting 5 folds for each of 4 candidates, totalling 20 fits


...

[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    4.2s remaining:    2.8s


[CV]  max_depth=4, n_estimators=126, max_features=10, min_samples_split=15, score=-0.03400562538575458, total=   0.4s
[CV]  max_depth=4, n_estimators=126, max_features=10, min_samples_split=15, score=-0.03548894059153004, total=   0.4s
[CV]  max_depth=2, n_estimators=2000, max_features=10, min_samples_split=2, score=-0.0389122267226357, total=   2.6s


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    6.2s finished


,mean_fit_time,mean_score_time,mean_test_score,param_max_depth,param_max_features,param_min_samples_split,param_n_estimators,params,rank_test_score,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,std_fit_time,std_score_time,std_test_score
24,0.459629,0.003212,-0.035623,2,10,2,216,"{'max_depth': 2, 'n_estimators': 216, 'max_fea...",1,-0.037123,-0.034057,-0.035793,-0.034577,-0.036563,0.018932,0.000963,0.001158
45,2.698534,0.008775,-0.038956,20,10,15,125,"{'max_depth': 20, 'n_estimators': 125, 'max_fe...",1,-0.041191,-0.037050,-0.040446,-0.036907,-0.039183,0.135284,0.001800,0.001738
40,1.464396,0.019186,-0.036177,2,2,15,1957,"{'max_depth': 2, 'n_estimators': 1957, 'max_fe...",1,-0.038176,-0.033416,-0.036005,-0.035464,-0.037822,0.137583,0.003811,0.001724
36,3.635270,0.022241,-0.036932,2,10,2,2000,"{'max_depth': 2, 'n_estimators': 2000, 'max_fe...",1,-0.039095,-0.034084,-0.037160,-0.036146,-0.038174,0.049800,0.010266,0.001732
32,0.219178,0.004110,-0.036261,3,3,2,83,"{'max_depth': 3, 'n_estimators': 83, 'max_feat...",1,-0.037592,-0.034944,-0.036607,-0.035389,-0.036770,0.011606,0.004550,0.000964
30,0.247045,0.001034,-0.036067,3,10,14,66,"{'max_depth': 3, 'n_estimators': 66, 'max_feat...",1,-0.037946,-0.034502,-0.036497,-0.034711,-0.036679,0.019638,0.000042,0.001294
20,0.427734,0.002380,-0.035090,3,10,15,149,"{'max_depth': 3, 'n_estimators': 149, 'max_fea...",1,-0.036712,-0.033295,-0.035433,-0.034054,-0.035956,0.022067,0.000336,0.001248
17,1.454455,0.016985,-0.035986,2,2,15,2000,"{'max_depth': 2, 'n_estimators': 2000, 'max_fe...",1,-0.038341,-0.033290,-0.035664,-0.035188,-0.037449,0.043686,0.001542,0.001772
12,0.263921,0.004275,-0.038158,20,2,15,50,"{'max_depth': 20, 'n_estimators': 50, 'max_fea...",1,-0.039947,-0.036482,-0.038443,-0.036626,-0.039291,0.013338,0.001280,0.001394
49,0.480261,0.001770,-0.035059,4,10,15,126,"{'max_depth': 4, 'n_estimators': 126, 'max_fea...",1,-0.036825,-0.033031,-0.035943,-0.034006,-0.035489,0.035490,0.000221,0.001365


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=4, max_features=10,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=15, min_weight_fraction_leaf=0.0,
             n_estimators=126, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False) 
Best score: -0.03505874605980852


In [111]:
print("Bandgap RMSLE: {:.6f}, formation RMSLE: {:.6f}".format(abs(rfr_g.best_score_), abs(rfr_f.best_score_)))
print("Average RMSLE: {:.6f}".format(abs((rfr_g.best_score_ + rfr_f.best_score_)/2)))

Bandgap RMSLE: 0.092875, formation RMSLE: 0.035045
Average RMSLE: 0.063960


In [112]:
## predictions with the best estimators
best_g = gbr_g.best_estimator_
best_f = gbr_f.best_estimator_

df_test['bandgap_energy_ev'] = best_g.fit(X, y_g).predict(X_test)
df_test['formation_energy_ev_natom'] = best_f.fit(X, y_f).predict(X_test)
gbr_pred = df_test[['formation_energy_ev_natom', 'bandgap_energy_ev']]

In [113]:
## predictions with the first ranking estimators

res_g = pd.DataFrame(gbr_g.cv_results_)
res_f = pd.DataFrame(gbr_f.cv_results_)
data_test = pd.read_csv('test_new_features.csv', index_col='id')

def predict(X, y_g, y_f, X_test, res_g, res_f):
    params_g = res_g.groupby('rank_test_score').get_group(1)
    params_f = res_f.groupby('rank_test_score').get_group(1)
                
    y_pred_g = np.array([]).reshape(0, X_test.shape[0])
    val_g = np.array([]).reshape(0, 10)
    for i, param in enumerate(params_g["params"]):
        best_g = GBR(**param).fit(X, y_g)
        y_pred_g = np.vstack([y_pred_g, best_g.predict(X_test)])
        val = cross_val_score(GBR(**param), X, y_g, scoring=my_score, cv=10)
        val_g = np.vstack([val_g, val])
    
    mean_cv_g = np.mean(val_g, axis=0)
    display(pd.DataFrame(y_pred_g))
    df_test['std_g'] = np.std(y_pred_g, axis=0)
    df_test['bandgap_energy_ev'] = np.mean(y_pred_g, axis=0)
             
    y_pred_f = np.array([]).reshape(0,X_test.shape[0])
    val_f = np.array([]).reshape(0, 10)
    for j, param in enumerate(params_f["params"]):
        best_f = GBR(**param).fit(X, y_f)
        y_pred_f = np.vstack([y_pred_f, best_f.predict(X_test)])
        val = cross_val_score(GBR(**param), X, y_f, scoring=my_score, cv=10)
        val_f = np.vstack([val_f, val])
            
    mean_cv_f = np.mean(val_f, axis=0)  
    display(pd.DataFrame(y_pred_f))
    df_test['std_f'] = np.std(y_pred_f, axis=0)
    df_test['formation_energy_ev_natom'] = np.mean(y_pred_f, axis=0)
        
    return df_test, df_test[['formation_energy_ev_natom', 'bandgap_energy_ev']]
        

pred = predict(X, y_g, y_f, X_test, res_g, res_f)
display(pred[0])

,0,1,2,3,4,5,6,7,8,9,...,590,591,592,593,594,595,596,597,598,599
0,1.572535,3.806902,3.523715,3.084594,1.611698,4.704049,3.240058,2.071921,2.725643,2.977289,...,2.894821,1.477997,1.048080,3.694278,1.677522,1.356569,0.956081,0.654443,2.381072,2.200894
1,1.569135,3.779830,3.503552,3.059327,1.608581,4.612575,3.233952,2.023207,2.793152,2.983419,...,2.905882,1.474163,1.045866,3.686244,1.560795,1.343939,0.943258,0.623642,2.249406,2.186218
2,1.565406,3.740797,3.464400,3.024320,1.609984,4.656290,3.240115,2.041913,2.827245,2.939607,...,2.897345,1.499690,1.088931,3.687946,1.599129,1.354741,0.923760,0.662126,2.303894,2.106889
3,1.576272,3.677313,3.470127,2.891393,1.613379,4.728619,3.282362,2.017353,2.785902,2.984405,...,2.888320,1.320950,1.034248,3.696573,1.574871,1.333938,0.935485,0.721468,2.143174,2.139328
4,1.555526,3.785677,3.485782,3.056630,1.650462,4.734011,3.231668,2.049716,2.836001,2.955348,...,2.902626,1.483560,1.051023,3.694732,1.514807,1.376314,0.944456,0.575980,2.302681,2.198109
5,1.601078,3.605427,3.406155,2.811888,1.620205,4.507499,3.359885,2.070300,2.692034,2.946110,...,2.813973,1.193219,0.976914,3.707638,1.629010,1.377671,0.967469,0.775316,2.118564,2.204640
6,1.562499,3.785384,3.488696,3.059528,1.650504,4.662087,3.234412,2.064938,2.874054,2.973518,...,2.908733,1.484086,1.066342,3.698181,1.565825,1.374785,0.937216,0.587657,2.309926,2.130931
7,1.615386,3.784513,3.477701,3.066112,1.652411,4.682608,3.235366,2.018092,2.864247,2.973097,...,2.901740,1.495408,1.074930,3.698416,1.579014,1.380262,0.945211,0.601629,2.284411,2.149120
8,1.570267,3.765123,3.495389,3.085435,1.651487,4.497281,3.232734,2.058523,2.859688,2.958897,...,2.914284,1.476627,1.091808,3.693415,1.587398,1.377350,0.937948,0.637004,2.275195,2.146515
9,1.599381,3.789323,3.510733,3.071069,1.633482,4.684203,3.238703,1.913553,2.804222,2.973651,...,2.895893,1.482421,1.055586,3.693497,1.561861,1.358371,0.945257,0.655391,2.357291,2.175608


,0,1,2,3,4,5,6,7,8,9,...,590,591,592,593,594,595,596,597,598,599
0,0.191007,0.057561,0.145470,0.025780,0.141683,0.076017,0.070788,0.129455,0.069820,0.249347,...,0.032482,0.211933,0.120292,0.063229,0.140787,0.032410,0.240033,0.176297,0.212750,0.214164
1,0.182392,0.062103,0.146723,0.026041,0.140812,0.088636,0.071093,0.131248,0.073196,0.248702,...,0.032750,0.210012,0.121476,0.063674,0.136878,0.033823,0.237677,0.163296,0.216084,0.210660
2,0.189708,0.074023,0.148903,0.032695,0.132811,0.057445,0.074217,0.121786,0.081455,0.241458,...,0.030381,0.210112,0.119126,0.062261,0.171366,0.046784,0.213200,0.156987,0.233930,0.212681
3,0.206640,0.063795,0.144835,0.025948,0.141475,0.126907,0.071567,0.134989,0.071666,0.249168,...,0.033924,0.211397,0.120796,0.064163,0.148858,0.032815,0.240021,0.159553,0.214755,0.213263
4,0.194356,0.057954,0.151031,0.031677,0.131718,0.042755,0.073223,0.129190,0.062859,0.243027,...,0.033410,0.209621,0.118994,0.062025,0.149887,0.035025,0.227470,0.161762,0.229908,0.212697
5,0.174151,0.060722,0.133316,0.034552,0.134349,0.072724,0.073721,0.129208,0.071358,0.246184,...,0.029485,0.202748,0.122548,0.065880,0.149904,0.039025,0.219294,0.180792,0.239999,0.207892
6,0.175591,0.070596,0.149897,0.029345,0.125033,0.069374,0.076709,0.126516,0.081782,0.252189,...,0.031385,0.200392,0.122547,0.065081,0.169896,0.046465,0.208516,0.177012,0.233891,0.209340
7,0.175720,0.074661,0.133067,0.038293,0.132105,0.079941,0.075625,0.138090,0.080930,0.240999,...,0.041785,0.207508,0.126651,0.067452,0.156358,0.060357,0.209270,0.184760,0.242512,0.202446
8,0.176692,0.079702,0.138568,0.040890,0.123586,0.076008,0.078778,0.118695,0.086597,0.239871,...,0.033085,0.211191,0.134398,0.062630,0.174739,0.059229,0.200430,0.200281,0.239603,0.210923
9,0.175615,0.062811,0.145578,0.026612,0.131937,0.042693,0.072665,0.119313,0.051311,0.248410,...,0.033262,0.213152,0.118993,0.063112,0.147865,0.042030,0.234770,0.174376,0.227434,0.205258


,sg,n_atoms,x_Al,x_Ga,x_In,a,b,c,alpha,beta,...,avg_IP,avg_LUMO,avg_mass,avg_rd_max,avg_rp_max,avg_rs_max,bandgap_energy_ev,formation_energy_ev_natom,std_g,std_f
id,,,,,,,,,,,,,,,,,,,,,
1,33,80.0,0.1875,0.4688,0.3438,10.5381,9.0141,9.6361,89.9997,90.0003,...,-0.499489,0.032240,6.748878,0.197757,0.120840,0.091832,1.577498,0.185125,0.018271,0.011178
2,33,80.0,0.7500,0.2500,0.0000,9.8938,8.5014,9.1298,90.0038,90.0023,...,-0.603143,0.032139,3.924055,0.299251,0.151839,0.114075,3.758084,0.064975,0.057047,0.006973
3,167,30.0,0.6667,0.1667,0.1667,4.9811,4.9808,13.4799,89.9900,90.0109,...,-0.595236,0.039659,5.049802,0.285567,0.150554,0.113952,3.481110,0.144095,0.029867,0.005560
4,12,80.0,0.5625,0.4375,0.0000,24.3370,6.0091,5.7620,89.9995,103.8581,...,-0.566828,0.025803,4.466880,0.263468,0.139404,0.104507,3.020102,0.030341,0.080671,0.004967
5,12,80.0,0.1875,0.5000,0.3125,24.6443,6.2906,6.1589,90.0000,104.5929,...,-0.495524,0.030406,6.562892,0.196483,0.119535,0.090693,1.633578,0.134029,0.019905,0.005762
6,33,40.0,1.0000,0.0000,0.0000,4.8885,8.4018,9.0344,90.0061,90.0024,...,-0.623075,0.039670,2.908569,0.335253,0.161698,0.121812,4.638767,0.069499,0.079698,0.023225
7,167,30.0,0.3333,0.6667,0.0000,4.9772,4.9768,13.4593,89.9952,90.0061,...,-0.603215,0.021750,5.764347,0.257334,0.144081,0.107714,3.249733,0.073761,0.034341,0.002337
8,33,80.0,0.1875,0.6875,0.1250,10.2955,8.8306,9.4505,89.9988,90.0014,...,-0.537799,0.022835,6.270590,0.215143,0.127503,0.095787,2.028908,0.127456,0.039756,0.005519
9,33,80.0,0.3125,0.6875,0.0000,10.0984,8.6759,9.2968,89.9995,90.0021,...,-0.570277,0.020073,5.536163,0.241308,0.135847,0.101532,2.809773,0.073149,0.052796,0.008800
